In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors
from scipy import ndarray

In [2]:
filename='preprocessed_majority.csv'
datapd_0=pd.read_csv(filename, index_col=0)

In [3]:
filename='preprocessed_minority.csv'
datapd_1=pd.read_csv(filename, index_col=0 )

In [4]:
print('Majority class dataframe shape:', datapd_0.shape)
print('Minority class dataframe shape:', datapd_1.shape)

Majority class dataframe shape: (16383, 99)
Minority class dataframe shape: (625, 99)


In [5]:
n_feat=datapd_0.shape[1]

In [6]:
print('Imbalance Ratio:', datapd_0.shape[0]/datapd_1.shape[0])

Imbalance Ratio: 26.2128


In [7]:
features_0=np.asarray(datapd_0)
features_1=np.asarray(datapd_1)

In [8]:
s=93
features_1=np.take(features_1,np.random.RandomState(seed=s).permutation(features_1.shape[0]),axis=0,out=features_1)
features_0=np.take(features_0,np.random.RandomState(seed=s).permutation(features_0.shape[0]),axis=0,out=features_0)

In [9]:
a=len(features_1)//3
b=len(features_0)//3

In [10]:
fold_1_min=features_1[0:a]
fold_1_maj=features_0[0:b]
fold_1_tst=np.concatenate((fold_1_min,fold_1_maj))
lab_1_tst=np.concatenate((np.zeros(len(fold_1_min))+1, np.zeros(len(fold_1_maj))))

In [11]:
fold_2_min=features_1[a:2*a]
fold_2_maj=features_0[b:2*b]
fold_2_tst=np.concatenate((fold_2_min,fold_2_maj))
lab_2_tst=np.concatenate((np.zeros(len(fold_1_min))+1, np.zeros(len(fold_1_maj))))

In [12]:
fold_3_min=features_1[2*a:]
fold_3_maj=features_0[2*b:]
fold_3_tst=np.concatenate((fold_3_min,fold_3_maj))
lab_3_tst=np.concatenate((np.zeros(len(fold_3_min))+1, np.zeros(len(fold_3_maj))))

In [13]:
fold_1_trn=np.concatenate((fold_2_min,fold_3_min,fold_2_maj,fold_3_maj))
lab_1_trn=np.concatenate((np.zeros(a+len(fold_3_min))+1,np.zeros(b+len(fold_3_maj))))

In [14]:
fold_2_trn=np.concatenate((fold_1_min,fold_3_min,fold_1_maj,fold_3_maj))
lab_2_trn=np.concatenate((np.zeros(a+len(fold_3_min))+1,np.zeros(b+len(fold_3_maj))))

In [15]:
fold_3_trn=np.concatenate((fold_2_min,fold_1_min,fold_2_maj,fold_1_maj))
lab_3_trn=np.concatenate((np.zeros(2*a)+1,np.zeros(2*b)))

In [16]:
training_folds_feats=[fold_1_trn,fold_2_trn,fold_3_trn]

In [17]:
testing_folds_feats=[fold_1_tst,fold_2_tst,fold_3_tst]

In [18]:
training_folds_labels=[lab_1_trn,lab_2_trn,lab_3_trn]

In [19]:
testing_folds_labels=[lab_1_tst,lab_2_tst,lab_3_tst]

In [20]:
def lr(X_train,y_train,X_test,y_test):
    from sklearn import metrics
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import balanced_accuracy_score
    logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial', class_weight={0: 1, 1: 1})
    logreg.fit(X_train, y_train)
    y_pred= logreg.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('balanced accuracy_LR:', bal_acc)
    print('f1 score_LR:', f1)
    print('confusion matrix_LR',con_mat)
    return(f1, bal_acc, precision, recall, con_mat)

In [21]:
def svm(X_train,y_train,X_test,y_test):
    from sklearn import preprocessing
    from sklearn import metrics
    #from sklearn import svm
    from sklearn.svm import LinearSVC
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import balanced_accuracy_score
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    #svm= svm.SVC(kernel='linear', decision_function_shape='ovo', class_weight={0: 1., 1: 1.},probability=True)
    svm= LinearSVC(random_state=0, tol=1e-5)
    svm.fit(X_train, y_train)
    y_pred= svm.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('balanced accuracy_SVM:', bal_acc)
    print('f1 score_SVM:', f1)
    print('confusion matrix_SVM',con_mat)
    return( f1, bal_acc, precision, recall, con_mat)

In [22]:
def knn(X_train,y_train,X_test,y_test):
    from sklearn import metrics
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score   
    from sklearn.metrics import balanced_accuracy_score
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train, y_train)
    y_pred= knn.predict(X_test)
    con_mat=confusion_matrix(y_test,y_pred)
    bal_acc=balanced_accuracy_score(y_test,y_pred)
    tn, fp, fn, tp = con_mat.ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    print('balanced accuracy_KNN:', bal_acc)
    f1 = f1_score(y_test, y_pred)
    precision=precision_score(y_test, y_pred)
    recall=recall_score(y_test, y_pred)
    print('f1 score_KNN:', f1)
    print('confusion matrix_KNN',con_mat)
    return(f1, bal_acc, precision, recall, con_mat)

In [23]:
def Neb_grps(data,near_neb):
    nbrs = NearestNeighbors(n_neighbors=near_neb, algorithm='ball_tree').fit(data)
    distances, indices = nbrs.kneighbors(data)
    neb_class=[]
    for i in (indices):
        neb_class.append(i)
    return(np.asarray(neb_class)) 

def LoRAS(data,num_samples,shadow,sigma,num_RACOS,num_afcomb):
    np.random.seed(42)
    data_shadow=([])
    for i in range (num_samples):
        c=0
        while c<shadow:
            data_shadow.append(data[i]+np.random.normal(0,sigma))
            c=c+1
    data_shadow==np.asarray(data_shadow)
    data_shadow_lc=([])
    for i in range(num_RACOS):
        idx = np.random.randint(shadow*num_samples, size=num_afcomb)
        w=np.random.randint(100, size=len(idx))
        aff_w=np.asarray(w/sum(w))
        data_tsl=np.array(data_shadow)[idx,:]
        data_tsl_=np.dot(aff_w, data_tsl)
        data_shadow_lc.append(data_tsl_)
    return(np.asarray(data_shadow_lc))   

def LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb):
    RACOS_set=[]
    for i in range (len(nb_list)):
        RACOS_i= LoRAS(features_1_trn[nb_list[i]],num_samples,shadow,sigma,num_RACOS,num_afcomb)
        RACOS_set.append(RACOS_i)
    LoRAS_set=np.asarray(RACOS_set)
    LoRAS_1=np.reshape(LoRAS_set,(len(features_1_trn)*num_RACOS,n_feat))
    return(np.concatenate((LoRAS_1,features_1_trn)))

In [24]:
def OVS(training_data,training_labels,neb):
    from imblearn.over_sampling import SMOTE 
    sm = SMOTE(random_state=62, k_neighbors=neb,  kind='regular',ratio=1)
    SMOTE_feat, SMOTE_labels = sm.fit_resample(training_data,training_labels)
    smbl1 = SMOTE(random_state=62, k_neighbors=neb,  kind='borderline1',ratio=1)
    SMOTE_feat_bl1, SMOTE_labels_bl1 = smbl1.fit_resample(training_data,training_labels)
    smbl2 = SMOTE(random_state=62, k_neighbors=neb,  kind='borderline2',ratio=1)
    SMOTE_feat_bl2, SMOTE_labels_bl2 = smbl2.fit_resample(training_data,training_labels)
    smsvm = SMOTE(random_state=62, k_neighbors=neb,  kind='svm',ratio=1)
    SMOTE_feat_svm, SMOTE_labels_svm = smsvm.fit_resample(training_data,training_labels)
    from imblearn.over_sampling import ADASYN
    ad = ADASYN(random_state=62,n_neighbors=neb,  ratio=1)
    ADASYN_feat, ADASYN_labels = ad.fit_resample(training_data,training_labels)
    return(SMOTE_feat, SMOTE_labels,SMOTE_feat_bl1, SMOTE_labels_bl1, SMOTE_feat_bl2, SMOTE_labels_bl2,SMOTE_feat_svm, SMOTE_labels_svm,ADASYN_feat, ADASYN_labels)

In [25]:
LR=[]
SVM=[]
KNN=[] 
LR_SM=[]
SVM_SM=[]
KNN_SM=[]
LR_SMBL1=[]
SVM_SMBL1=[]
KNN_SMBL1=[] 
LR_SMBL2=[]
SVM_SMBL2=[]
KNN_SMBL2=[] 
LR_SMSVM=[]
SVM_SMSVM=[]
KNN_SMSVM=[] 
LR_ADA=[]
SVM_ADA=[]
KNN_ADA=[] 

i=0
while i<3:
    SMOTE_feat, SMOTE_labels,SMOTE_feat_bl1, SMOTE_labels_bl1, SMOTE_feat_bl2, SMOTE_labels_bl2,SMOTE_feat_svm, SMOTE_labels_svm,ADASYN_feat, ADASYN_labels=OVS(training_folds_feats[i],training_folds_labels[i],30)
    
    f1_lr, bal_acc_lr, precision_lr, recall_lr, mat_lr=lr(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    LR.append([f1_lr, bal_acc_lr, precision_lr, recall_lr])
    f1_svm,bal_acc_svm,precision_svm, recall_svm,mat_svm=svm(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    SVM.append([f1_svm,bal_acc_svm,precision_svm, recall_svm])
    f1_knn,bal_acc_knn,precision_knn, recall_knn,mat_knn=knn(training_folds_feats[i],training_folds_labels[i],testing_folds_feats[i],testing_folds_labels[i])
    KNN.append([f1_knn,bal_acc_knn,precision_knn, recall_knn])
    
    f1_lr_SMOTE,bal_acc_lr_SMOTE,precision_lr_SMOTE, recall_lr_SMOTE,mat_lr_SMOTE=lr(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_SM.append([f1_lr_SMOTE,bal_acc_lr_SMOTE,precision_lr_SMOTE, recall_lr_SMOTE])
    f1_svm_SMOTE,bal_acc_svm_SMOTE,precision_svm_SMOTE, recall_svm_SMOTE,mat_svm_SMOTE=svm(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SM.append([f1_svm_SMOTE,bal_acc_svm_SMOTE,precision_svm_SMOTE, recall_svm_SMOTE])
    f1_knn_SMOTE,bal_acc_knn_SMOTE,precision_knn_SMOTE, recall_knn_SMOTE,mat_knn_SMOTE=knn(SMOTE_feat,SMOTE_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SM.append([f1_knn_SMOTE,bal_acc_knn_SMOTE,precision_knn_SMOTE, recall_knn_SMOTE])
    
    f1_lr_SMOTE_bl1,bal_acc_lr_SMOTE_bl1,precision_lr_SMOTE_bl1, recall_lr_SMOTE_bl1,mat_lr_SMOTE_bl1=lr(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMBL1.append([f1_lr_SMOTE_bl1,bal_acc_lr_SMOTE_bl1,precision_lr_SMOTE_bl1, recall_lr_SMOTE_bl1])
    f1_svm_SMOTE_bl1,bal_acc_svm_SMOTE_bl1,precision_svm_SMOTE_bl1, recall_svm_SMOTE_bl1,mat_svm_SMOTE_bl1=svm(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMBL1.append([f1_svm_SMOTE_bl1,bal_acc_svm_SMOTE_bl1,precision_svm_SMOTE_bl1, recall_svm_SMOTE_bl1])
    f1_knn_SMOTE_bl1,bal_acc_knn_SMOTE_bl1,precision_knn_SMOTE_bl1, recall_knn_SMOTE_bl1,mat_knn_SMOTE_bl1=knn(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMBL1.append([f1_knn_SMOTE_bl1,bal_acc_knn_SMOTE_bl1,precision_knn_SMOTE_bl1, recall_knn_SMOTE_bl1])
    
    
    f1_lr_SMOTE_bl2,bal_acc_lr_SMOTE_bl2,precision_lr_SMOTE_bl2, recall_lr_SMOTE_bl2,mat_lr_SMOTE_bl2=lr(SMOTE_feat_bl2,SMOTE_labels_bl2,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMBL2.append([f1_lr_SMOTE_bl2,bal_acc_lr_SMOTE_bl2,precision_lr_SMOTE_bl2, recall_lr_SMOTE_bl2])
    f1_svm_SMOTE_bl2,bal_acc_svm_SMOTE_bl2,precision_svm_SMOTE_bl2, recall_svm_SMOTE_bl2,mat_svm_SMOTE_bl2=svm(SMOTE_feat_bl1,SMOTE_labels_bl1,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMBL2.append([f1_svm_SMOTE_bl2,bal_acc_svm_SMOTE_bl2,precision_svm_SMOTE_bl2, recall_svm_SMOTE_bl2])
    f1_knn_SMOTE_bl2,bal_acc_knn_SMOTE_bl2,precision_knn_SMOTE_bl2, recall_knn_SMOTE_bl2,mat_knn_SMOTE_bl2=knn(SMOTE_feat_bl2,SMOTE_labels_bl2,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMBL2.append([f1_knn_SMOTE_bl2,bal_acc_knn_SMOTE_bl2,precision_knn_SMOTE_bl2, recall_knn_SMOTE_bl2])
    
    f1_lr_SMOTE_svm,bal_acc_lr_SMOTE_svm,precision_lr_SMOTE_svm, recall_lr_SMOTE_svm,mat_lr_SMOTE_svm=lr(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    LR_SMSVM.append([f1_lr_SMOTE_svm,bal_acc_lr_SMOTE_svm,precision_lr_SMOTE_svm, recall_lr_SMOTE_svm])
    f1_svm_SMOTE_svm,bal_acc_svm_SMOTE_svm,precision_svm_SMOTE_svm, recall_svm_SMOTE_svm,mat_svm_SMOTE_svm=svm(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    SVM_SMSVM.append([f1_svm_SMOTE_svm,bal_acc_svm_SMOTE_svm,precision_svm_SMOTE_svm, recall_svm_SMOTE_svm])
    f1_knn_SMOTE_svm,bal_acc_knn_SMOTE_svm,precision_knn_SMOTE_svm, recall_knn_SMOTE_svm,mat_knn_SMOTE_svm=knn(SMOTE_feat_svm,SMOTE_labels_svm,testing_folds_feats[i],testing_folds_labels[i])
    KNN_SMSVM.append([f1_knn_SMOTE_svm,bal_acc_knn_SMOTE_svm,precision_knn_SMOTE_svm, recall_knn_SMOTE_svm])
    
    f1_lr_ADASYN,bal_acc_lr_ADASYN,precision_lr_ADASYN, recall_lr_ADASYN,mat_lr_ADASYN=lr(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_ADA.append([f1_lr_ADASYN,bal_acc_lr_ADASYN,precision_lr_ADASYN, recall_lr_ADASYN])
    f1_svm_ADASYN,bal_acc_svm_ADASYN,precision_svm_ADASYN, recall_svm_ADASYN,mat_svm_ADASYN=svm(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_ADA.append([f1_svm_ADASYN,bal_acc_svm_ADASYN,precision_svm_ADASYN, recall_svm_ADASYN])
    f1_knn_ADASYN,bal_acc_knn_ADASYN,precision_knn_ADASYN, recall_knn_ADASYN,mat_knn_ADASYN=knn(ADASYN_feat,ADASYN_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_ADA.append([f1_knn_ADASYN,bal_acc_knn_ADASYN,precision_knn_ADASYN, recall_knn_ADASYN])
    
    i=i+1

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in

tn, fp, fn, tp: 5438 23 31 177
balanced accuracy_LR: 0.923374927809784
f1 score_LR: 0.8676470588235293
confusion matrix_LR [[5438   23]
 [  31  177]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 5439 22 30 178
balanced accuracy_SVM: 0.9258703322862818
f1 score_SVM: 0.8725490196078431
confusion matrix_SVM [[5439   22]
 [  30  178]]
tn, fp, fn, tp: 5450 11 42 166
balanced accuracy_KNN: 0.8980313199892948
f1 score_KNN: 0.8623376623376624
confusion matrix_KNN [[5450   11]
 [  42  166]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

tn, fp, fn, tp: 4062 1399 2 207
balanced accuracy_SVM: 0.8671252176152955
f1 score_SVM: 0.22809917355371898
confusion matrix_SVM [[4062 1399]
 [   2  207]]
tn, fp, fn, tp: 5343 118 7 202
balanced accuracy_KNN: 0.972449706443866
f1 score_KNN: 0.7637051039697542
confusion matrix_KNN [[5343  118]
 [   7  202]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


tn, fp, fn, tp: 5303 158 18 191
balanced accuracy_LR: 0.9424715840641207
f1 score_LR: 0.6845878136200717
confusion matrix_LR [[5303  158]
 [  18  191]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 3911 1550 2 207
balanced accuracy_SVM: 0.8532999108949146
f1 score_SVM: 0.21057985757884026
confusion matrix_SVM [[3911 1550]
 [   2  207]]
tn, fp, fn, tp: 5271 190 7 202
balanced accuracy_KNN: 0.9658575072129559
f1 score_KNN: 0.6722129783693843
confusion matrix_KNN [[5271  190]
 [   7  202]]


In [26]:
LR_LoRAS=[]
SVM_LoRAS=[]
KNN_LoRAS=[]
for i in range(3):
    
    features = training_folds_feats[i]
    labels= training_folds_labels[i]
    label_1=np.where(labels == 1)[0]
    label_1=list(label_1)
    features_1_trn=features[label_1]
    
    label_0=np.where(labels == 0)[0]
    label_0=list(label_0)
    features_0_trn=features[label_0]
    
    num_samples=30 
    shadow=100
    sigma=.005
    num_RACOS=(len(features_0_trn)-len(features_1_trn))//len(features_1_trn)
    num_afcomb=50
    nb_list=Neb_grps(features_1_trn, num_samples)
    
    LoRAS_1=LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb)
    LoRAS_train=np.concatenate((LoRAS_1,features_0_trn))
    LoRAS_labels=np.concatenate((np.zeros(len(LoRAS_1))+1, np.zeros(len(features_0_trn))))
    
    f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS,mat_lr_LoRAS=lr(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_LoRAS.append([f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS])
    f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS,mat_svm_LoRAS=svm(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_LoRAS.append([f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS])
    f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS,mat_knn_LoRAS=knn(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_LoRAS.append([f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS])    

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


tn, fp, fn, tp: 5428 33 23 185
balanced accuracy_LR: 0.941690113814038
f1 score_LR: 0.8685446009389671
confusion matrix_LR [[5428   33]
 [  23  185]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 3993 1468 3 205
balanced accuracy_SVM: 0.858380843886017
f1 score_SVM: 0.21796916533758637
confusion matrix_SVM [[3993 1468]
 [   3  205]]
tn, fp, fn, tp: 5369 92 11 197
balanced accuracy_KNN: 0.9651343266237515
f1 score_KNN: 0.7927565392354124
confusion matrix_KNN [[5369   92]
 [  11  197]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

tn, fp, fn, tp: 4090 1371 1 207
balanced accuracy_SVM: 0.8720696934909076
f1 score_SVM: 0.23180291153415455
confusion matrix_SVM [[4090 1371]
 [   1  207]]
tn, fp, fn, tp: 5373 88 7 201
balanced accuracy_KNN: 0.9751159445297424
f1 score_KNN: 0.8088531187122736
confusion matrix_KNN [[5373   88]
 [   7  201]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


tn, fp, fn, tp: 5418 43 25 184
balanced accuracy_LR: 0.936254379685793
f1 score_LR: 0.8440366972477065
confusion matrix_LR [[5418   43]
 [  25  184]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 4081 1380 2 207
balanced accuracy_SVM: 0.8688648257456746
f1 score_SVM: 0.2305122494432071
confusion matrix_SVM [[4081 1380]
 [   2  207]]
tn, fp, fn, tp: 5344 117 10 199
balanced accuracy_KNN: 0.9653642312736945
f1 score_KNN: 0.758095238095238
confusion matrix_KNN [[5344  117]
 [  10  199]]


In [27]:
LR_tLoRAS=[]
SVM_tLoRAS=[]
KNN_tLoRAS=[]
from sklearn.manifold import TSNE
for i in range(3):
    
    features = training_folds_feats[i]
    labels= training_folds_labels[i]
    label_1=np.where(labels == 1)[0]
    label_1=list(label_1)
    features_1_trn=features[label_1]
    
    label_0=np.where(labels == 0)[0]
    label_0=list(label_0)
    features_0_trn=features[label_0]
    
    data_embedded_min = TSNE().fit_transform(features_1_trn)
    result_min= pd.DataFrame(data = data_embedded_min, columns = ['t-SNE0', 't-SNE1'])
    min_t=np.asmatrix(result_min)
    min_t=min_t[0:len(features_1_trn)]
    min_t=min_t[:, [0,1]]
    
    num_samples=30 
    shadow=100
    sigma=.005
    num_RACOS=(len(features_0_trn)-len(features_1_trn))//len(features_1_trn)
    num_afcomb=50
    nb_list=Neb_grps(min_t, num_samples)
    
    LoRAS_1=LoRAS_gen(num_samples,shadow,sigma,num_RACOS,num_afcomb)
    LoRAS_train=np.concatenate((LoRAS_1,features_0_trn))
    LoRAS_labels=np.concatenate((np.zeros(len(LoRAS_1))+1, np.zeros(len(features_0_trn))))
    
    f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS,mat_lr_LoRAS=lr(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    LR_tLoRAS.append([f1_lr_LoRAS,bal_acc_lr_LoRAS,precision_lr_LoRAS, recall_lr_LoRAS])
    f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS,mat_svm_LoRAS=svm(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    SVM_tLoRAS.append([f1_svm_LoRAS,bal_acc_svm_LoRAS,precision_svm_LoRAS, recall_svm_LoRAS])
    f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS,mat_knn_LoRAS=knn(LoRAS_train,LoRAS_labels,testing_folds_feats[i],testing_folds_labels[i])
    KNN_tLoRAS.append([f1_knn_LoRAS,bal_acc_knn_LoRAS,precision_knn_LoRAS, recall_knn_LoRAS])    

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


tn, fp, fn, tp: 5426 35 24 184
balanced accuracy_LR: 0.9391031510148888
f1 score_LR: 0.8618266978922716
confusion matrix_LR [[5426   35]
 [  24  184]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 4014 1447 3 205
balanced accuracy_SVM: 0.8603035686616991
f1 score_SVM: 0.22043010752688172
confusion matrix_SVM [[4014 1447]
 [   3  205]]
tn, fp, fn, tp: 5382 79 10 198
balanced accuracy_KNN: 0.9687284309720676
f1 score_KNN: 0.8164948453608246
confusion matrix_KNN [[5382   79]
 [  10  198]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

tn, fp, fn, tp: 4118 1343 1 207
balanced accuracy_SVM: 0.8746333265251504
f1 score_SVM: 0.2354948805460751
confusion matrix_SVM [[4118 1343]
 [   1  207]]
tn, fp, fn, tp: 5375 86 8 200
balanced accuracy_KNN: 0.9728952150211994
f1 score_KNN: 0.8097165991902834
confusion matrix_KNN [[5375   86]
 [   8  200]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


tn, fp, fn, tp: 5412 49 24 185
balanced accuracy_LR: 0.9380973742474914
f1 score_LR: 0.8352144469525958
confusion matrix_LR [[5412   49]
 [  24  185]]


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


tn, fp, fn, tp: 4114 1347 3 206
balanced accuracy_SVM: 0.8694939058955675
f1 score_SVM: 0.23382519863791143
confusion matrix_SVM [[4114 1347]
 [   3  206]]
tn, fp, fn, tp: 5354 107 11 198
balanced accuracy_KNN: 0.9638874700026021
f1 score_KNN: 0.7704280155642024
confusion matrix_KNN [[5354  107]
 [  11  198]]


In [28]:
def stats(arr):
    x=np.mean(np.asarray(arr), axis = 0)
    y=np.std(np.asarray(arr), axis = 0)
    return(x,y)

In [29]:
print('F1|Balanced Accuracy|precision|recall :: mean|sd')
print('Without Oversampling')

LR_m, LR_sd=stats(LR)
print('lr:',LR_m, LR_sd)
SVM_m, SVM_sd=stats(SVM)
print('svm:',SVM_m, SVM_sd)
KNN_m, KNN_sd= stats(KNN)
print('knn:',KNN_m, KNN_sd)

print('SMOTE Oversampling')
LR_SM_m, LR_SM_sd=stats(LR_SM)
print('lr:',LR_SM_m, LR_SM_sd)
SVM_SM_m, SVM_SM_sd=stats(SVM_SM)
print('svm:',SVM_SM_m, SVM_SM_sd)
KNN_SM_m, KNN_SM_sd=stats(KNN_SM)
print('knn:',KNN_SM_m, KNN_SM_sd)

print('SMOTE-Bl1 Oversampling')
LR_SMBL1_m, LR_SMBL1_sd=stats(LR_SMBL1)
print('lr:',LR_SMBL1_m, LR_SMBL1_sd)
SVM_SMBL1_m,SVM_SMBL1_sd=stats(SVM_SMBL1)
print('svm:',SVM_SMBL1_m,SVM_SMBL1_sd)
KNN_SMBL1_m,  KNN_SMBL1_sd= stats(KNN_SMBL1)
print('knn:',KNN_SMBL1_m,  KNN_SMBL1_sd)

print('SMOTE-Bl2 Oversampling')
LR_SMBL2_m, LR_SMBL2_sd=stats(LR_SMBL2)
print('lr:',LR_SMBL2_m, LR_SMBL2_sd)
SVM_SMBL2_m, SVM_SMBL2_sd=stats(SVM_SMBL2)
print('svm:',SVM_SMBL2_m, SVM_SMBL2_sd)
KNN_SMBL2_m, KNN_SMBL2_sd= stats(KNN_SMBL2)
print('knn:',KNN_SMBL2_m, KNN_SMBL2_sd)

print('SMOTE-SVM Oversampling')
LR_SMSVM_m, LR_SMSVM_sd=stats(LR_SMSVM)
print('lr:',LR_SMSVM_m, LR_SMSVM_sd)
SVM_SMSVM_m, SVM_SMSVM_sd=stats(SVM_SMSVM)
print('svm:',SVM_SMSVM_m, SVM_SMSVM_sd)
KNN_SMSVM_m, KNN_SMSVM_sd= stats(KNN_SMSVM)
print('knn:',KNN_SMSVM_m, KNN_SMSVM_sd)

print('ADASYN Oversampling')
LR_ADA_m, LR_ADA_sd=stats(LR_ADA)
print('lr:',LR_ADA_m, LR_ADA_sd)
SVM_ADA_m, SVM_ADA_sd=stats(SVM_ADA)
print('svm:',SVM_ADA_m, SVM_ADA_sd)
KNN_ADA_m, KNN_ADA_sd=stats(KNN_ADA)
print('knn:',KNN_ADA_m, KNN_ADA_sd)

print('LoRAS Oversampling')
LR_LoRAS_m, LR_LoRAS_sd=stats(LR_LoRAS)
print('lr:',LR_LoRAS_m, LR_LoRAS_sd)
SVM_LoRAS_m, SVM_LoRAS_sd=stats(SVM_LoRAS)
print('svm:',SVM_LoRAS_m, SVM_LoRAS_sd)
KNN_LoRAS_m, KNN_LoRAS_sd=stats(KNN_LoRAS)
print('knn:',KNN_LoRAS_m, KNN_LoRAS_sd)

print('tLoRAS Oversampling')
LR_tLoRAS_m, LR_tLoRAS_sd=stats(LR_tLoRAS)
print('lr:',LR_tLoRAS_m, LR_tLoRAS_sd)
SVM_tLoRAS_m, SVM_tLoRAS_sd=stats(SVM_tLoRAS)
print('svm:',SVM_tLoRAS_m, SVM_tLoRAS_sd)
KNN_tLoRAS_m, KNN_tLoRAS_sd=stats(KNN_tLoRAS)
print('knn:',KNN_tLoRAS_m, KNN_tLoRAS_sd)

F1|Balanced Accuracy|precision|recall :: mean|sd
Without Oversampling
lr: [0.86697083 0.92268866 0.88512568 0.84958901] [0.01060415 0.00341861 0.01539276 0.00624579]
svm: [0.86614949 0.91279481 0.90766798 0.82882468] [0.0048657  0.00927121 0.01388145 0.01911944]
knn: [0.86908916 0.89768859 0.95602148 0.79678107] [0.00792244 0.00510041 0.01285659 0.01023042]
SMOTE Oversampling
lr: [0.76449654 0.95442821 0.65086196 0.92802264] [0.02569505 0.00518219 0.03703568 0.0101942 ]
svm: [0.1748892  0.81697422 0.09592629 0.99200252] [0.00949262 0.01248916 0.00567663 0.0022556 ]
knn: [0.78056364 0.97041168 0.65867719 0.95998957] [0.02190077 0.003221   0.03515312 0.00910366]
SMOTE-Bl1 Oversampling
lr: [0.74643789 0.95179182 0.62677972 0.92482517] [0.03034254 0.00609206 0.04149589 0.01112581]
svm: [0.22916108 0.86762094 0.12960958 0.98879739] [0.00590668 0.00481567 0.00375947 0.00227729]
knn: [0.74708726 0.97209861 0.60840924 0.96800239] [0.01062672 0.00114243 0.0144035  0.00222938]
SMOTE-Bl2 Oversamp